# Parameter Sweep Analysis

This notebook demonstrates comprehensive parameter space exploration using vectorized operations for efficient computation.

## Objectives

1. Perform multi-dimensional parameter sweeps
2. Visualize parameter sensitivity
3. Identify critical parameter regimes
4. Generate heatmaps and contour plots

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import LogNorm
import seaborn as sns

sys.path.insert(0, '..')

from quantro_simulator import (
    SimulationConfig,
    ModelType,
    OverlayMode,
    run_parameter_sweep,
    run_comprehensive_simulation
)

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

# Set random seed for reproducibility
np.random.seed(42)

## 1. Single Parameter Sweep - SIR Model

Let's examine how lambda affects the SIR model across different overlay modes:

In [ ]:
lambda_values = np.linspace(0.0, 1.0, 50)
modes = [OverlayMode.BASELINE, OverlayMode.RESIDUAL, OverlayMode.PARAM_MOD, OverlayMode.CONTROL]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, mode in enumerate(modes):
    config = SimulationConfig(
        model=ModelType.SIR,
        overlay_mode=mode,
        lambda_values=lambda_values
    )
    
    results = run_parameter_sweep(config)
    
    lambda_arr = np.array([r.lambda_param for r in results])
    val1_arr = np.array([r.val1 for r in results])
    val2_arr = np.array([r.val2 for r in results])
    
    ax = axes[idx]
    ax.plot(lambda_arr, val1_arr, 'o-', label='Mean (val1)', linewidth=2, markersize=4)
    ax.fill_between(lambda_arr, val1_arr - val2_arr, val1_arr + val2_arr, 
                     alpha=0.3, label='±1 Std Dev')
    ax.set_xlabel('Lambda Parameter', fontsize=11)
    ax.set_ylabel('Susceptible Population', fontsize=11)
    ax.set_title(f'{mode.value} Mode', fontsize=12, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../artifacts/parameter_sweep_sir.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ SIR parameter sweep completed")

## 2. Multi-Model Comparison

Compare how different models respond to parameter variations:

In [ ]:
models = [ModelType.MICHAELIS_MENTEN, ModelType.SIR, 
          ModelType.FITZHUGH_NAGUMO, ModelType.POISEUILLE]
lambda_values = np.linspace(0.0, 1.0, 30)

fig, ax = plt.subplots(figsize=(12, 7))

colors = ['blue', 'red', 'green', 'purple']
markers = ['o', 's', '^', 'd']

for model_type, color, marker in zip(models, colors, markers):
    config = SimulationConfig(
        model=model_type,
        overlay_mode=OverlayMode.RESIDUAL,
        lambda_values=lambda_values
    )
    
    results = run_parameter_sweep(config)
    
    lambda_arr = np.array([r.lambda_param for r in results])
    val1_arr = np.array([r.val1 for r in results])
    
    # Normalize to [0, 1] for comparison
    val1_norm = (val1_arr - val1_arr.min()) / (val1_arr.max() - val1_arr.min() + 1e-10)
    
    ax.plot(lambda_arr, val1_norm, marker=marker, label=model_type.value, 
            color=color, linewidth=2, markersize=5, alpha=0.8)

ax.set_xlabel('Lambda Parameter', fontsize=13)
ax.set_ylabel('Normalized Response', fontsize=13)
ax.set_title('Multi-Model Parameter Sensitivity (Residual Mode)', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../artifacts/multimodel_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Multi-model comparison completed")

## 3. 2D Parameter Space Exploration

Explore the interaction between lambda and time parameters:

In [ ]:
lambda_range = np.linspace(0.0, 1.0, 20)
t_end_range = np.linspace(5.0, 20.0, 20)

# Create meshgrid
Lambda, Tend = np.meshgrid(lambda_range, t_end_range)
Z_mean = np.zeros_like(Lambda)
Z_final = np.zeros_like(Lambda)

# Sweep through parameter space
print("Running 2D parameter sweep...")
for i, t_end in enumerate(t_end_range):
    config = SimulationConfig(
        model=ModelType.FITZHUGH_NAGUMO,
        overlay_mode=OverlayMode.TIME_WARP,
        t_end=t_end,
        lambda_values=lambda_range
    )
    
    results = run_parameter_sweep(config)
    
    for j, result in enumerate(results):
        Z_mean[i, j] = result.val1
        Z_final[i, j] = result.val3

# Create heatmaps
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Heatmap 1: Mean values
im1 = ax1.contourf(Lambda, Tend, Z_mean, levels=20, cmap='viridis')
ax1.set_xlabel('Lambda Parameter', fontsize=12)
ax1.set_ylabel('Simulation Duration (t_end)', fontsize=12)
ax1.set_title('Mean Membrane Potential', fontsize=13, fontweight='bold')
cbar1 = plt.colorbar(im1, ax=ax1)
cbar1.set_label('Mean Value', fontsize=11)

# Heatmap 2: Final values
im2 = ax2.contourf(Lambda, Tend, Z_final, levels=20, cmap='plasma')
ax2.set_xlabel('Lambda Parameter', fontsize=12)
ax2.set_ylabel('Simulation Duration (t_end)', fontsize=12)
ax2.set_title('Final State Value', fontsize=13, fontweight='bold')
cbar2 = plt.colorbar(im2, ax=ax2)
cbar2.set_label('Final Value', fontsize=11)

plt.tight_layout()
plt.savefig('../artifacts/2d_parameter_space.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ 2D parameter space exploration completed")

## 4. Statistical Analysis of Parameter Sensitivity

Quantify parameter sensitivity using statistical measures:

In [ ]:
# Generate comprehensive dataset
lambda_values = np.linspace(0.0, 1.0, 100)

sensitivity_data = []

for model_type in ModelType:
    config = SimulationConfig(
        model=model_type,
        overlay_mode=OverlayMode.PARAM_MOD,
        lambda_values=lambda_values
    )
    
    results = run_parameter_sweep(config)
    
    val1_arr = np.array([r.val1 for r in results])
    
    # Compute sensitivity metrics
    gradient = np.gradient(val1_arr)
    sensitivity = np.std(gradient)
    range_span = np.ptp(val1_arr)  # peak-to-peak
    
    sensitivity_data.append({
        'Model': model_type.value,
        'Gradient Std': sensitivity,
        'Value Range': range_span,
        'Max Gradient': np.max(np.abs(gradient))
    })

# Create DataFrame and visualize
df_sensitivity = pd.DataFrame(sensitivity_data)
print("\nParameter Sensitivity Analysis:")
print(df_sensitivity.to_string(index=False))

# Bar plot
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

metrics = ['Gradient Std', 'Value Range', 'Max Gradient']
for idx, metric in enumerate(metrics):
    ax = axes[idx]
    df_sensitivity.plot(x='Model', y=metric, kind='bar', ax=ax, 
                        color='steelblue', legend=False)
    ax.set_ylabel(metric, fontsize=11)
    ax.set_xlabel('Model', fontsize=11)
    ax.set_title(f'{metric} by Model', fontsize=12, fontweight='bold')
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../artifacts/sensitivity_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Statistical sensitivity analysis completed")

## 5. Export Results for Further Analysis

Save parameter sweep results to CSV for external analysis:

In [ ]:
# Run comprehensive simulation
print("Running comprehensive simulation across all models and modes...")
results_df = run_comprehensive_simulation()

# Save to CSV
output_path = '../parameter_sweep_results.csv'
results_df.to_csv(output_path, index=False)

print(f"\n✓ Results saved to {output_path}")
print(f"  Total simulations: {len(results_df)}")
print(f"\nDataset summary:")
print(results_df.describe())

# Group statistics
print("\nResults by model:")
print(results_df.groupby('model')['val1'].agg(['mean', 'std', 'min', 'max']))

## Summary

This notebook demonstrated:

✓ Single and multi-parameter sweeps  
✓ Cross-model comparisons  
✓ 2D parameter space visualization  
✓ Quantitative sensitivity analysis  
✓ Data export for reproducibility  

**Next**: Explore notebook 03 for deep-dive FitzHugh-Nagumo analysis.